### This notebook will calculate contacts within distance cutoff (say 10A) in each frame and record the distances too, and finally obtain a contact matrix. These will be required for the next notebook.

In [ ]:
import sys, os
import subprocess as sp
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
from matplotlib import colors
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('png')
plt.figure(figsize=(5,7))
import ipywidgets as widgets

## Running VMD
### Collect inputs for running  VMD

This section first collect inputs to run VMD job that calculates the contact data. 
<br>1. Working directory: Directory that contains toppar and trajectory (*.dcd) files. Output will also be written in this directory.
<br>2. Name of toppar and dcd files (with extension)
<br>3. Two selection between which the contacts will get calculated. If the selctions are same, all intra-selection contacts will be calculated.
<br>4. Cutoff in Angstrom for finding contacts
<br>5. Location of Required tcl scripts: vmd_contact.tcl, bigdcd.tcl [these files will be copied to the working directory]
<br> ** vmd_contact.tct [Adopted from https://www.ks.uiuc.edu/Research/vmd/mailing_list/vmd-l/att-5681/newcontacts.tcl to report inter-residue contacts in each frame of a trajectory]
<br> ** bigdcd.tcl [This script by Justin Gullingsrud and Axel Kohlmeyer (https://www.ks.uiuc.edu/Research/vmd/script_library/scripts/bigdcd/) helps analyze large trajectory files using VMD]
<br>6. Full path to the vmd executable

In [ ]:
class data_input():
    def __init__(self, 
                 pwd = "/../../../", 
                 top = "*.prmtop or *.psf", 
                 dcd = "*.dcd",
                 sele1 = "all and noh",
                 sele2 = "all and noh",
                 cutoff = "4.5",
                 tcl_loc = "path to tcl files (bigdcd.tcl and vmd_contact.tcl)",
                 vmd_path = "full path to vmd executable"
                ):
        layout = widgets.Layout(width='auto', height='40px') #set width and height
        self.pwd = widgets.Text(description = 'Working Dir:',value = pwd, layout = layout)
        self.top = widgets.Text(description = 'Toppar file:',value = top, layout = layout)
        self.dcd = widgets.Text(description = 'Traj:',value = dcd, layout = layout) 
        self.sele1 = widgets.Text(description = 'selection-1:',value = sele1, layout = layout) 
        self.sele2 = widgets.Text(description = 'selection-2:',value = sele2, layout = layout)
        self.cutoff = widgets.Text(description = 'Cutoff (A):',value = cutoff, layout = layout)
        self.tcl_loc = widgets.Text(description = 'Tcl path:',value = tcl_loc, layout = layout)
        self.vmd_path = widgets.Text(description = 'VMD path:',value = vmd_path, layout = layout)
        self.pwd.on_submit(self.handle_submit)
        self.top.on_submit(self.handle_submit)
        self.dcd.on_submit(self.handle_submit)
        self.sele1.on_submit(self.handle_submit)
        self.sele2.on_submit(self.handle_submit)
        self.cutoff.on_submit(self.handle_submit)
        self.tcl_loc.on_submit(self.handle_submit)
        self.vmd_path.on_submit(self.handle_submit)
        display(self.pwd, self.top, self.dcd, self.sele1, self.sele2, self.cutoff, self.tcl_loc, self.vmd_path)

    def handle_submit(self, text):
        self.v = text.value
        return self.v

print("After input, press return in any field")
f = data_input()

#### Set path for the tcl files and copy to the working directory . . .

In [ ]:
tcl_files = os.path.join(f.tcl_loc.value, "*.tcl")
# print(tcl_files)
copy = 'cp '+tcl_files+" "+f.pwd.value+"."
# print(copy)
sp.run(copy, shell=True)

#### Check values of the given inputs . . .
If found discrepancy, check cell-2

In [ ]:
print("Working Dir:", f.pwd.value)
print("toppar file:", f.top.value)
print("DCD file:", f.dcd.value)
print("Selection 1:", f.sele1.value)
print("Selection 2:", f.sele2.value)
print("Cutoff distance (Angstrom):", f.cutoff.value)
print("List of Content in the working directory", os.listdir(f.pwd.value))

#### Change to working directory . . .
Create the contact_final.tcl with the parsed inputs for calculation of contacts..

In [ ]:
print("Current directory:", os.getcwd())
if os.getcwd()+"/" != f.pwd.value:
    os.chdir(f.pwd.value)
    print("\nNow in working directory:", os.getcwd())
else:
    print("\nAlready in working directory ....")
sed = 'sed '+'\"s/top_file/'+f.top.value+'/g; s/dcd_file/'+f.dcd.value+'/g; s/cutoff_dist/'+f.cutoff.value+'/g; s/selection1/'+f.sele1.value+'/g; s/selection2/'+f.sele2.value+'/g\" '+'vmd_contact.tcl >'+' contact_final.tcl'
print(sed)
sp.run(sed, shell=True)

#### Check for contact_all.dat (output of the VMD run) from any previous run . . . 
* contact_all.dat: 1 line per contact and frame
* format: frame number, contacting residues, minimal distance

Delete if exists, otherwise output of the current run will get appended in the previous file..

In [ ]:
if os.path.exists("contact_all.dat"):
  copy = 'cp '+"contact_all.dat"+" "+"contact_all.bak.dat"
  sp.run(copy, shell=True)
  os.remove("contact_all.dat")
  print("Backup created and deleted contact_all.dat file from previous run ....\n")
else:
  print("The file does not exist\n")

print("List of Content in the working directory", os.listdir(f.pwd.value))

### Run VMD
Output: 
1. contact_all.dat: 
<br> 1 line per contact per frame 
<br> Fields: frame number, contacting residues, minimal distance
<br> example:
<br> Say, if in frame 1, residue 361 (THR) is in contact with residue 363 (ASP) with minimal distance ~3.2 A
<br> 1  361  363  3.197465
2. Contact-out.log & contact-err.log for stdout & stderr respectively
<br>
3. During the run, update of contact_all.dat can be checked by running the following command in the shell: "tail -f contact_all.dat"

In [ ]:
outlogFile = open("Contact-out.log", "w")
errlogFile = open("Contact-err.log", "w")
# p = sp.Popen(['/Applications/VMD_1.9.4a51-x86_64-Rev9.app/Contents/Resources/VMD.app/Contents/MacOS/VMD', '-dispdev', 'none', '-e', 'contact_final.tcl'], stdout=outlogFile, stdin=sp.PIPE, stderr=errlogFile)
p = sp.Popen([f.vmd_path.value, '-dispdev', 'none', '-e', 'contact_final.tcl'], stdout=outlogFile, stdin=sp.PIPE, stderr=errlogFile)
# p.communicate()
p.wait()
if p.poll is None:
    print("VMD still running..")
else:
    print("Run finished.. check whether the output contact_all.dat is okay")

## Calculate Contact Map
### Collect inputs
1. Working directoy: Directory that contains contact_all.dat (from VMD run). This directory will also be used as output directory.
<br>
2. Prefix of the output contact map
<br>
** Even if desired selection length of 1st group of residue is different than the 2nd group, its best to put the whole residue range (of the complex) for both selection. This is because its more flexible and easier to handle  the contact matrix later to get contact information of selected regions. For e.g., if I need to see contacts between resid 200-400 and resid 700-800, I provide 1 to 1200 for both "res_range1" and "res_range2" (see following keys) which is basically the total residue range for my whole complex.

In [ ]:
class req_inputs():
    def __init__(self, 
                 pwd1 = "/../../../", 
                 out = "prefix of output map",
                 res_range1 = "provide the range of residues from selection1 [e.g. 1 to 1200]",
                 res_range2 = "provide the range of residues from selection2 [e.g. 1 to 1200]"
                ):
        layout = widgets.Layout(width='auto', height='40px') #set width and height
        self.pwd1 = widgets.Text(description = 'Working Dir:',value = pwd1, layout = layout)
        self.out = widgets.Text(description = 'Outfile prefix:',value = out, layout = layout)
        self.res_range1 = widgets.Text(description = 'Res_range1:',value = res_range1, layout = layout)
        self.res_range2 = widgets.Text(description = 'Res_range2:',value = res_range2, layout = layout)
        self.pwd1.on_submit(self.handle_submit)
        self.out.on_submit(self.handle_submit)
        self.res_range1.on_submit(self.handle_submit)
        self.res_range2.on_submit(self.handle_submit)
        display(self.pwd1, self.out, self.res_range1, self.res_range2)

    def handle_submit(self, text):
        self.v = text.value
        return self.v

print("After input, press return in any field")
g = req_inputs()

### Function to calculate the contact matrix  

In [ ]:
def save_contact_map(contact_file, res_range1, res_range2, frames, output_file):
    f2 = 1
    running_contact = np.zeros(shape=(res_range1, res_range2))
    t = np.zeros(shape=(res_range1, res_range2))

    with open(contact_file, "r") as input:
        for line in input:
            lines = line.split()
            f = int(lines[0])
            res1 = int(lines[1])
            res2 = int(lines[2])
                
            f1 = f
            if f1 == f2:  # Avoid repeating same contacts from same frames
                running_contact[res1, res2] = 1
                if res_range1 == res_range2:
                    running_contact[res2, res1] = 1
            else:
                t = np.add(t, running_contact)  # Updating after each frame
                running_contact = np.zeros(shape=(res_range1, res_range2))
                running_contact[res1, res2] = 1
                if res_range1 == res_range2:
                    running_contact[res2, res1] = 1
            f2 = f1

    t = np.add(t, running_contact)

    contactMat = np.true_divide(t, frames)  # Getting the fraction

    if res_range1 == res_range2:
        for x in range(res_range1):
            contactMat[x, x] = 1  # Same residue contact
            for y in range(x, res_range1):
                contactMat[x, y] = contactMat[y, x]  # For the lower diagonal

    print("Saving the map in a tabular format...\n")
    print("Original residue IDs are maintained as provided in the selection....\n")

    with open(output_file + ".dat", "w+") as fo2:
        for i in range(res_range1):
            for j in range(res_range2):
                if contactMat[i, j] != 0:
                    fo2.write("%s %s %8.3f\n" % (i + int(g.res_range1.value.split()[0]), j + (int(g.res_range2.value.split()[0])), contactMat[i, j]))

    print("Done Saving.. check the *.dat file\n")

    print("Saving the map in *.npy format...\n")
    print("It's just the matrix, so only indexes for the provided range start from 0\n")
    print("Be careful if you are plotting the matrix directly, you have to change the indexes accordingly..\n")

    np.save(output_file + ".npy", contactMat)
    print("Done Saving..\n")

    print("Printing the contact map...")
    print(contactMat)  # Check cmap


### Get frame id and resid pairs for calculation of contact map and get total residues, total frames . . .

In [ ]:
contact = os.path.join(g.pwd1.value, "contact_all.dat")
res_range1 = len(np.arange(int(g.res_range1.value.split()[0]), int(g.res_range1.value.split()[2])+1))
res_range2 = len(np.arange(int(g.res_range2.value.split()[0]), int(g.res_range2.value.split()[2])+1))
print("Selection-1 length:",res_range1,"\n", "Selection-2 length:", res_range2, "\n")

s4 = 'tail -n1 '+contact+' | awk \'{print $1}\''
frames=int(sp.getoutput(s4)) # Get total frames
print('No of frames:',frames)

Calculate the contact map from contact data:

In [ ]:
outfile = os.path.join(g.pwd1.value, g.out.value)

# Get the contact map without any exclusion
save_contact_map(contact, res_range1, res_range2, frames, outfile)

In [ ]:
# Renaming the generic output name for feeding into the 2nd notebook
! mv contact_all.dat contact_all_tgRNA_HEPN2_20A_sample.dat 